Import our needed libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

Request the website and then parse the website using Beautiful Soup

In [2]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup=BeautifulSoup(source, 'lxml')

Grab the desired table and put data in to dataframe

In [4]:
table = soup.find_all('table')[0] # Grab the first table

In [5]:
data=[]
for tr in soup.find_all('table')[0]('tr'):
    td=tr.find_all('td')
    row=[tr.text for tr in td]
    data.append(row)
post_df = pd.DataFrame(data,columns=["Postcode","Borough","Neighborhood"])

Converted "Borough" value "Not Assigned" to NaN and then removed them

In [6]:
post_df['Neighborhood'] = post_df.Neighborhood.str.replace(r'\n', '')
post_df['Borough']=post_df['Borough'].replace('Not assigned', np.nan)
post_df=post_df.dropna(subset = ['Borough'])

Any Neighborhood with "Not Assigned" value was assigned the same value as the Borough on the same row

In [7]:
post_df.loc[post_df['Neighborhood'] == "Not assigned",'Neighborhood'] = post_df['Borough']

Grouped Neighborhoods with similar Postcodes

In [8]:
post1_df = post_df.groupby(['Postcode'])['Neighborhood'].apply(', '.join).reset_index()
del post_df['Neighborhood']
post_df=pd.merge(post_df, post1_df, on='Postcode')
post_df.head(5)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [9]:
post_df.drop_duplicates(subset='Postcode', keep="last")
post_df.shape

(210, 3)

End of 1st Problem

2nd Problem - Import Latitude and Longitude as CSV file

In [10]:
import types
import pandas as pd
from botocore.client import Config 
import ibm_boto3 

def __iter__(self): return 0 
# @hidden_cell 
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials. 
# You might want to remove those credentials before you share the notebook. 
client_62f881625bff4148851d2b2de1bf71b8 = ibm_boto3.client(service_name='s3',
        ibm_api_key_id='axnRTH0Q0vGQvl785SfLMobtBO6Tj8m3RqwV8w6kymei', 
        ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token", 
        config=Config(signature_version='oauth'), 
        endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com') 

body = client_62f881625bff4148851d2b2de1bf71b8.get_object(Bucket='torontoneighborhoodclustering-donotdelete-pr-5lmfdvxmrejdzt',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body ) 
    
lat_df = pd.read_csv(body)
lat_df.head() 

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge Latitude and Longitude with Postcode/Borough/Neighborhood data for Toronto

In [12]:
lat_df.rename(columns={'Postal Code':'Postcode'}, inplace=True)
postfinal_df = pd.merge(post_df, lat_df, on='Postcode')
postfinal_df.head(5)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763


End of 2nd Problem


3rd problem - Download all needed libraries

In [13]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Calculate Latitude & Longitude for Toronto and generate a map

In [14]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(postfinal_df['Latitude'], postfinal_df['Longitude'], postfinal_df['Borough'], postfinal_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Select Borough of Downtown Toronto

In [16]:
downtown_data = postfinal_df[postfinal_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


Calculate Latitude and Longitude of Downtown Toronto & Generate Map

In [17]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))





The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [18]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Generate Four Square ID data

In [19]:
CLIENT_ID = '2PFXVNLDPPS5FZPT1PPS0QIG0KLG05G540EVJXOALLEF4POQ' # your Foursquare ID
CLIENT_SECRET = '3FMS4VJFQO3XEHSEXA2MSVP3RWN2D53GNXYPPMDC5BXRXVID' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2PFXVNLDPPS5FZPT1PPS0QIG0KLG05G540EVJXOALLEF4POQ
CLIENT_SECRET:3FMS4VJFQO3XEHSEXA2MSVP3RWN2D53GNXYPPMDC5BXRXVID


Select 2nd Neighborhood in Downtown Toronto

In [20]:
downtown_data.loc[1, 'Neighborhood']

"Queen's Park"

In [22]:
neighborhood_latitude = downtown_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Queen's Park are 43.6623015, -79.3894938.


Query venues within 1000m of Queen's Park

In [23]:
# type your answer here
limit=15
radius=1000

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude,limit, radius)
url


'https://api.foursquare.com/v2/venues/explore?client_id=2PFXVNLDPPS5FZPT1PPS0QIG0KLG05G540EVJXOALLEF4POQ&client_secret=3FMS4VJFQO3XEHSEXA2MSVP3RWN2D53GNXYPPMDC5BXRXVID&v=20180605&ll=43.6623015,-79.3894938&limit=15&radius=1000'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2493b00f5968002240cb01'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 43.67130150900001,
    'lng': -79.37707613733237},
   'sw': {'lat': 43.653301490999986, 'lng': -79.40191146266763}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b9d206bf964a520e69136e3',
       'name': "Queen's Park",
       'location': {'address': 'University Ave.',
        'crossStreet': 'at Wellesley Ave.',
        'lat': 43.66394609897775,
        'lng': -79.39217952520835,
       

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Nando's,Portuguese Restaurant,43.661728,-79.386391
2,Mercatto,Italian Restaurant,43.660391,-79.387664
3,YMCA,Gym,43.662753,-79.384849
4,T-Swirl Crepe,Creperie,43.663452,-79.384125


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

15 venues were returned by Foursquare.


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
downtown = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Adelaide, King, Richmond
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Harbourfront East, Toronto Islands, Union Station
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Chinatown, Grange Park, Kensington Market
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South

Check the size of the resulting dataframe

In [30]:
print(downtown.shape)
downtown.head()

(544, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


How many venues for each neighborhood

In [31]:
downtown.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",45,45,45,45,45,45
Berczy Park,15,15,15,15,15,15
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",105,105,105,105,105,105
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,15,15,15,15,15,15
"Chinatown, Grange Park, Kensington Market",45,45,45,45,45,45
Christie,15,15,15,15,15,15
Church and Wellesley,15,15,15,15,15,15
"Commerce Court, Victoria Hotel",30,30,30,30,30,30


In [32]:
print('There are {} uniques categories.'.format(len(downtown['Venue Category'].unique())))

There are 104 uniques categories.


Analyze the different venues in each neighborhood

In [33]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head(5)

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Ethiopian Restaurant,Farmers Market,Food Truck,Fountain,French Restaurant,Gas Station,Gastropub,General Entertainment,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Seafood Restaurant,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
downtown_onehot.shape

(544, 104)

Print each neighborhood with the top 5 venues in each neighborhood

In [35]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0  Asian Restaurant  0.13
1             Plaza  0.07
2        Steakhouse  0.07
3  Greek Restaurant  0.07
4       Opera House  0.07


----Berczy Park----
                venue  freq
0      Farmers Market  0.13
1   French Restaurant  0.07
2        Concert Hall  0.07
3              Museum  0.07
4  Seafood Restaurant  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0     Airport Service  0.20
1      Airport Lounge  0.13
2    Airport Terminal  0.13
3             Airport  0.07
4  Airport Food Court  0.07


----Cabbagetown, St. James Town----
                 venue  freq
0                 Café  0.13
1        Deli / Bodega  0.07
2    Indian Restaurant  0.07
3   Italian Restaurant  0.07
4  Japanese Restaurant  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.33
1  Italian Restaurant  0.07
2     Bubbl

Create dataframe to house the top 5 venues data

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Asian Restaurant,Café,American Restaurant,Plaza,Coffee Shop,Hotel,Speakeasy,Steakhouse,Concert Hall,Pizza Place
1,Berczy Park,Farmers Market,Fountain,Vegetarian / Vegan Restaurant,Museum,Park,Thai Restaurant,Tea Room,French Restaurant,Concert Hall,Seafood Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina,Bar
3,"Cabbagetown, St. James Town",Café,Bakery,General Entertainment,Diner,Deli / Bodega,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Caribbean Restaurant
4,Central Bay Street,Coffee Shop,Park,Sushi Restaurant,Modern European Restaurant,Bubble Tea Shop,Ramen Restaurant,Japanese Restaurant,Seafood Restaurant,Italian Restaurant,Gastropub


Use clusters to examine the neighborhood with 5 clusters

In [38]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 0, 3, 0, 0, 1, 0, 0], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Breakfast Spot,Performing Arts Venue,Dessert Shop,Park,Chocolate Shop,Pub,Restaurant,Spa,Bakery
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,3,Coffee Shop,Park,Yoga Studio,Arts & Crafts Store,Burrito Place,Portuguese Restaurant,Mexican Restaurant,Beer Bar,Italian Restaurant,Creperie
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Café,Music Venue,Taco Place,Comic Shop,Plaza,Clothing Store,Pizza Place,Ramen Restaurant,Electronics Store,Tea Room
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Café,Music Venue,Taco Place,Comic Shop,Plaza,Clothing Store,Pizza Place,Ramen Restaurant,Electronics Store,Tea Room
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Coffee Shop,Japanese Restaurant,Cosmetics Shop,BBQ Joint,Café,Gym,Food Truck,Creperie,Restaurant


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters